# HOW TO MASK THE NOT ARRIVED JOBS  in DJSSP Environment
- for that overwrite ``_get_job_machine_availability(td)`` method


## Random Job Arrivals are generated by the DJSSPGenerator
- We dont have to change anything in ``_reset`` or `_step`

In [1]:
import torch

from rl4co.envs.scheduling.djssp.env import DJSSPEnv

generator_params = {"num_jobs":5 ,
    "num_machines": 7 ,
    "min_processing_time":1 ,
    "max_processing_time": 200 ,
    "mtbf" : 17 ,
    "mttr" : 4}
djssp_env = DJSSPEnv(generator_params=generator_params)

C:\Users\soner\anaconda3\envs\env_rl4co\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
td = djssp_env.reset(batch_size = [3])
td

IN BATCH 0 JOB WITH ID 2 not arrived yet
IN BATCH 0 JOB WITH ID 3 not arrived yet
IN BATCH 0 JOB WITH ID 4 not arrived yet
IN BATCH 1 JOB WITH ID 2 not arrived yet
IN BATCH 1 JOB WITH ID 3 not arrived yet
IN BATCH 1 JOB WITH ID 4 not arrived yet
IN BATCH 2 JOB WITH ID 2 not arrived yet
IN BATCH 2 JOB WITH ID 3 not arrived yet
IN BATCH 2 JOB WITH ID 4 not arrived yet


TensorDict(
    fields={
        action_mask: Tensor(shape=torch.Size([3, 6]), device=cpu, dtype=torch.bool, is_shared=False),
        actual_proc_times: Tensor(shape=torch.Size([3, 7, 35]), device=cpu, dtype=torch.float32, is_shared=False),
        adjacency: Tensor(shape=torch.Size([3, 35, 35]), device=cpu, dtype=torch.float32, is_shared=False),
        busy_until: Tensor(shape=torch.Size([3, 7]), device=cpu, dtype=torch.float32, is_shared=False),
        done: Tensor(shape=torch.Size([3, 1]), device=cpu, dtype=torch.bool, is_shared=False),
        end_op_per_job: Tensor(shape=torch.Size([3, 5]), device=cpu, dtype=torch.int64, is_shared=False),
        finish_times: Tensor(shape=torch.Size([3, 35]), device=cpu, dtype=torch.float32, is_shared=False),
        is_ready: Tensor(shape=torch.Size([3, 35]), device=cpu, dtype=torch.bool, is_shared=False),
        job_arrival_times: Tensor(shape=torch.Size([3, 5]), device=cpu, dtype=torch.float32, is_shared=False),
        job_done: Tensor(sh

In [3]:
td["job_arrival_times"]

tensor([[  8.5715,  32.8602,  43.9085,  60.7955,  71.4690],
        [ 23.8151,  32.2920,  84.4821, 147.1975, 158.8463],
        [  4.2324,  25.0350,  55.0256,  56.7638, 138.2026]])

## ``_get_job_machine_availability`` example !Without JOB ARRIVAL MASK!
- return: ACTION MASK containing 0/1 - False/ True
    - [batch_size , number of jobs , number of machines]
    - According to my understanding:
        - FALSE : Machine can operate next operation of the job
        - TRUE : Machine can NOT operate this operation

In [4]:
action_mask = djssp_env._get_job_machine_availability(td)
action_mask

tensor([[[False,  True,  True,  True,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True, False,  True,  True,  True],
         [False,  True,  True,  True,  True,  True,  True]],

        [[ True,  True, False,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True, False,  True,  True,  True],
         [ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True,  True,  True, False,  True]],

        [[ True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True,  True,  True,  True, False],
         [ True,  True,  True, False,  True,  True,  True]]])

In [5]:
action_mask.shape

torch.Size([3, 5, 7])

In [6]:
# in batch 0 : x-achse = Machine IDX y-achse = job_idx
action_mask[0]

tensor([[False,  True,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True],
        [ True,  True,  True, False,  True,  True,  True],
        [False,  True,  True,  True,  True,  True,  True]])

-  in batch 2 machine 1 can not process the opeation of the job 2

In [7]:

#           bs  job machine
action_mask [2] [2] [1]

tensor(True)

- However according to job arrival times at the current time (0) job 2 is not yet arrived

In [8]:
# TODO: Why td["time"] has 3 elements ?
td["job_arrival_times"][2][2]<=td["time"]

tensor([False, False, False])

## Before adding dynamic job arrivals the output of the ``get_action_mask(td)` method is as following:
- They filled the job index 0 with False -> it is like padding
    - They have started counting job id's starting from 1
    - in batch 0 job1-job2-job3-job4-job 5 has feasible actions

In [5]:
djssp_env.get_action_mask(td)

tensor([[False,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True],
        [False,  True,  True,  True,  True,  True]])

# ``get_job_machine_availability(td)`` with the JOB ARRIVAL
- td["job_arrival_times"] has following structure
  - [batch_no][arr_time_job1, arr_time_job2, ........] and my code works as follows
    - Iterate over each batch
        - Iterate over each job
                - if job arrival time in current batch <= current time
                        - boo.add true
                - else
                        - boo.add false
                    - at the end boo has following structure
                            - torch.Tensor([True, True, False, False])
                                    - meaning job 0-1 arrived
                                            - job 2-3 not arrived
                - check the boolean value in boo
                        - if false: fill the whole job with true (meaning not available)
                        - otherwise job-machine affiliation stays same

In [4]:
action_mask = djssp_env._get_job_machine_availability(td)
action_mask

JOB WITH ID 0 not arrived yet
JOB WITH ID 1 not arrived yet
JOB WITH ID 2 not arrived yet
JOB WITH ID 3 not arrived yet
JOB WITH ID 4 not arrived yet
JOB WITH ID 0 not arrived yet
JOB WITH ID 1 not arrived yet
JOB WITH ID 2 not arrived yet
JOB WITH ID 3 not arrived yet
JOB WITH ID 4 not arrived yet
JOB WITH ID 0 not arrived yet
JOB WITH ID 1 not arrived yet
JOB WITH ID 2 not arrived yet
JOB WITH ID 3 not arrived yet
JOB WITH ID 4 not arrived yet


tensor([[[True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True]],

        [[True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True],
         [True, True, True, True, True, True, True]]])

In [5]:
action_mask.shape

torch.Size([3, 5, 7])

In [6]:
# in batch 0 : x-achse = Machine IDX y-achse = job_idx
action_mask[0]

tensor([[True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True]])

# Currently it is all set to True(meaning none of the jobs is arrived )
- that is because we didnot take step and advance the time
    - meaning because of td["time"] = 0 , 0, 0

- For the test purposes i manually set the time in the method to 42
    - meaning in batch 0
        − job 0 -1 arrived -> false
        - others not arrived -> filled with true

In [6]:
action_mask = djssp_env._get_job_machine_availability(td)
action_mask

IN BATCH 0 JOB WITH ID 2 not arrived yet
IN BATCH 0 JOB WITH ID 3 not arrived yet
IN BATCH 0 JOB WITH ID 4 not arrived yet
IN BATCH 1 JOB WITH ID 2 not arrived yet
IN BATCH 1 JOB WITH ID 3 not arrived yet
IN BATCH 1 JOB WITH ID 4 not arrived yet
IN BATCH 2 JOB WITH ID 2 not arrived yet
IN BATCH 2 JOB WITH ID 3 not arrived yet
IN BATCH 2 JOB WITH ID 4 not arrived yet


tensor([[[ True,  True,  True, False,  True,  True,  True],
         [ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True]],

        [[ True,  True,  True, False,  True,  True,  True],
         [ True,  True,  True,  True, False,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True]],

        [[ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True,  True,  True, False,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True],
         [ True,  True,  True,  True,  True,  True,  True]]])

In [8]:
# in batch 0 : x-achse = Machine IDX y-achse = job_idx
action_mask[0]

tensor([[ True,  True,  True, False,  True,  True,  True],
        [ True,  True,  True,  True,  True, False,  True],
        [ True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True],
        [ True,  True,  True,  True,  True,  True,  True]])

In [9]:
djssp_env.get_action_mask(td)
# in (0,y) = False padding
# in all batched 1-2 is set true meaning JO-J1 in action_mask
# and others are set the false
# that is correct since this two jobs arrived
# anothers are not arrived
# and in the action_mask provided from get_job_machine_Availability j0-j1 has one false -> meaning feasible
# and other are filled with true -> meaning not feasible  

IN BATCH 0 JOB WITH ID 2 not arrived yet
IN BATCH 0 JOB WITH ID 3 not arrived yet
IN BATCH 0 JOB WITH ID 4 not arrived yet
IN BATCH 1 JOB WITH ID 2 not arrived yet
IN BATCH 1 JOB WITH ID 3 not arrived yet
IN BATCH 1 JOB WITH ID 4 not arrived yet
IN BATCH 2 JOB WITH ID 2 not arrived yet
IN BATCH 2 JOB WITH ID 3 not arrived yet
IN BATCH 2 JOB WITH ID 4 not arrived yet


tensor([[False,  True,  True, False, False, False],
        [False,  True,  True, False, False, False],
        [False,  True,  True, False, False, False]])